# Error Handling

See: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Error  
See: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Statements/try...catch  

## Error Handling Keywords

* There are four keywords used in error handling:
    - The ```try``` keyword tests a block of code for errors
    - The ```catch``` keyword handles any error that results in the ```try``` block
    - The ```finally``` keyword executes regardless of what happens in ```try``` and ```catch```
    - The ```throw``` keyword throws an error such as ```Error```, ```EvalError```, etc.
    
## What Can Be Thrown?

* Any datatype (```string``` or ```number```, etc.) can be thrown but ```Error``` types are most common
* The ```Error``` built-in type is used to encapsulate a general runtime error
* Other more specialized built-in error types exist (see below)
* The ```Error``` object can also be used as a base object for custom user-defined exception types

## ECMAScript Exceptions

ECMAScript exceptions are the standard built-in JavaScript error types:
* ```Error``` is the generic base type used for general purpose errors
* ```EvalError``` is an error in the ```eval()``` function 
* ```InternalError``` is an error thrown by the JavaScript engine (e.g. "too much recursion", etc.)
* ```RangeError``` is an error when a numeric variable or parameter is outside of valid range
* ```ReferenceError``` is an error when de-referencing an invalid reference
* ```SyntaxError``` is a error related to invalid syntax
* ```TypeError``` is an error when a variable or parameter is not of valid type
* ```URIError``` is an error when invalid parameters are passed to ```encodeURI()``` or ```decodeURI()```

## Browser Exceptions

JavaScript running in the browser can throw DOM-related errors (not supported in Node.js)
* ```DOMException``` can be thrown by calling methods  or accessing properties in the DOM API
* ```DOMError``` **NOTE: deprecated** (no longer supported/recommended)

In [53]:
try {
    throw "This is just a primitive string value" // usually you would throw and Error object
} catch (e) {
    console.error(e)                              // This is just a primitive string value
}

This is just a primitive string value


## Error Instance Members

* The ```Error.prototype.message``` property is the human readable error message text
* The ```Error.prototype.name``` property is the name of the Error
* Several additional non-standard browser-specific Error instance properties exist
* The ```Error.prototype.toString()``` method returns a string representing the specified object.

## Throwing and Catching Errors

Here is the syntax for creating and throwing a new ```Error``` object:

```
throw new Error([message[, fileName[, lineNumber]]])
```

* ```messageOptional``` is a human-readable string that describes the error
* ```fileName``` (optional) is the fileName property on the newly created Error object
    - Defaults to name of file that created the Error object
* ```lineNumber``` (optional) is the lineNumber property on the newly created Error object
    - Defaults to line number that created the Error object

This is how you use it:

* You define a block of code that contains code that may throw an error using the ```try``` keyword
* You can create an ```Error``` object and then raise the error using the ```throw``` keyword
* You catch the error using the ```catch``` keyword
* You can handle specific error types by testing the error type with the ```instanceof``` keyword

In [13]:
{
console.log("Antecedent code runs normally"); //Antecedent code runs normally
try {
    console.error("Starting try block")       // Starting try block
    throw new Error('Oops!')                  // this knocks us out of try block (usually in if)
    console.error("Ending try block")         // Unreachable code (never get here)
} catch (e) {
    console.error("Starting catch block")     // Starting catch block
    console.error(e.name)                     // Error
    console.error(e.message)                  // Oops!
    console.error(e.stack)                    // at evalmachine.<anonymous>:5:11 ... etc.
    console.error("Ending catch block")       // Ending catch block
} finally {
    console.error('Executing finally');       // Executing finally
}
console.log("Subsequent code runs normally"); // Subsequent code runs normally
}

Antecedent code runs normally


Starting try block
Starting catch block
Error
Oops!
Error: Oops!
    at evalmachine.<anonymous>:5:11
    at Script.runInThisContext (vm.js:96:20)
    at Object.runInThisContext (vm.js:303:38)
    at run ([eval]:1054:15)
    at onRunRequest ([eval]:888:18)
    at onMessage ([eval]:848:13)
    at process.emit (events.js:182:13)
    at emit (internal/child_process.js:812:12)
    at process._tickCallback (internal/process/next_tick.js:63:19)
Ending catch block
Executing finally


Subsequent code runs normally


## Handeling Specific Errors

* You can filter for specific types of error by using the ```instanceof``` operator
* In the code below, try commenting/uncommenting individual calls to ```foo()``` to see this work

In [16]:
function foo(n) {
    if( !(n >= 10 && n <= 20) ) {
        throw new RangeError("The argument n must be between 1 and 10.")
    }
    if( n === 13) {
        throw new Error('Oops!')
    }
    console.log("Smooth sailing :)");
}

try {
    // try commenting and uncommenting various combinations of the following three statements
    foo(15);                 // 15 -> Smooth sailing :)
    //foo(13);                 // 13 -> Error: Oops!
    //foo(23);                 // 23 -> RangeError: The argument n must be between 1 and 10.
} catch (e) {
    if (e instanceof RangeError) {
        console.error(e.name + ': ' + e.message)
    } else if (e instanceof Error) {
        console.error(e.name + ': ' + e.message)
    } else if (e instanceof RangeError) {
        console.error(e.name + ': ' + e.message)
    }
// ... etc
}
console.log("NOTE: Try calling foo() with different numeric parameters to see how instnceof is used");

Smooth sailing :)
NOTE: Try calling foo() with different numeric parameters to see how instnceof is used


## Catching Errors Up the Call Stack

* The ```try``` and ```catch``` can be in a different stack frame from where the error is thrown
* The following demo shows how an error is caught in code that calls a function that throws the error
* Of course, this works for any depth of calls that make calls to other functions
* The error is thrown and the error handling mechanism unwinds the stack until a catch is found
* The first catch that is found is executed and then it is done (no more than one catch executes)
* If no error handler is found, the runtime default error handler writes error info to console output

In [6]:
{
function foo(param) {
    if (param === "good") {
        console.log(param);
        return;
    } else {
        throw new Error('Oops!')
    }
}
    
try {
    foo("good");
    foo("bad");
} catch (err) {
    console.error(err.name + ': ' + err.message)
} finally {
    console.error('all done :)');
}
}

good


Error: Oops!
all done :)


## User Defined Exceptions

* The following demo shows a custom error named ```MustNotBeMultipleOfThreeError```
* The ```testMustNotBeMultipleOfThreeError``` function receives n and throws error when ```n%3===0```
* This function is called in a loop with values of ```n``` equal to whole numbers from 0 to 9
* The output shows that the error is thrown only for the values 0, 3, 6, 9 (multiples of 3)

In [47]:
{
class MustNotBeMultipleOfThreeError extends Error { // custom error class
  constructor(message) {
    super(message);
    this.name = "MustNotBeMultipleOfThreeError";
  }
}

function testMustNotBeMultipleOfThreeError(n) {    // function that may throw custom error object
    try {
        if (n%3===0) {
            throw new MustNotBeMultipleOfThreeError("n is a multiple of 3");
        }
    }
    catch (err) {
        console.error(err.toString());  // MustNotBeMultipleOfThreeError: n is a multiple of 3
    }
}

let numbers0To9 = [...Array(10).keys()] // short for Array.from(Array(10).keys())
for (let n of numbers0To9) {
    console.log("n: ", n);
    testMustNotBeMultipleOfThreeError(n);
}
}

n:  0


MustNotBeMultipleOfThreeError: n is a multiple of 3


n:  1
n:  2
n:  3


MustNotBeMultipleOfThreeError: n is a multiple of 3


n:  4
n:  5
n:  6


MustNotBeMultipleOfThreeError: n is a multiple of 3


n:  7
n:  8
n:  9


MustNotBeMultipleOfThreeError: n is a multiple of 3


## Catching the ```DOMException``` Error Object

* The ```DOMException``` object encapsulates errors in the browser DOM API
* The following demo calls ```Document.createAttribute()``` to create a new attribute node
* If you try to create an attribute with an invalid name it throws a ```DOMException``` error
* Buttons are provided to create attributes (one with valid name and one with invalid name)
* You can see the success/failure of creating the new attribute in the F12 Dev Tools console
* When the attribute is created successfully and applied to the ```div```, it's CSS color change to red
* To reset back to the original CSS color of the ```div```, just refresh your browser
* Note: The CSS may be messed up in Jupyter notebook but you could copy/paste into html file and view it directly in your web browser as a stand-alone web page

Copy/paste the following demo into an HTML file and open it in your browser to see it work

```html
<html>
<head>
    <script type="text/javascript">
        function mayThrowDOMException (attributeName) {
            try {
                var node = document.getElementById("div1");
                // following throws DOMException error if name of new attribute is invalid
                var attrib = document.createAttribute(attributeName);
                attrib.value = "***my cool attribute value***";
                node.setAttributeNode(attrib);
                console.log(node.getAttribute(attributeName));
            }
            catch (e) {
                if (e.code == DOMException.INVALID_CHARACTER_ERR) {
                    console.log(attributeName, "Bad: invalid attribute name");
                }
                return;
            }
            // If we get here then there was no error thrown
            console.log(attributeName, "Good: valid attribute name");
        }
    </script>
    <style>
        div[foo123] {color:red;} /* attribute  selector */
    </style>
</head>
<body>
    <p>Open browser F12 Dev Tools to see <code>console.log()</code> output</p>
    <p>Click "Throw DOM Exception" and see console "123 Bad: invalid attribute name" (*** div color unchanged)</p>
    <p>Click "Don't Throw DOM Exception" and see console "foo123 Good: valid attribute name" (*** div color should change to red as long as other css ru;es that me be active do not interfere with it)"</p>
    <p>To reset the div red css style back to original black, just refresh your browser</p>
    <button onclick="mayThrowDOMException ('123')">Throw DOM Exception</button>
    <button onclick="mayThrowDOMException ('foo123')">Don't Throw DOM Exception</button>
    <div id="div1">***</div>
</body>
</html>
```